In [1]:
import os
import pandas as pd
import numpy as np
import scipy
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import KFold, train_test_split
from datetime import datetime
from utils_glenn import *  # Assuming this file contains necessary utility functions

def create_run_directory():
    base_dir = "runs"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

class CustomTensorBoardCallback(tf.keras.callbacks.Callback):
    def __init__(self, log_dir, fold, total_folds):
        super().__init__()
        self.log_dir = log_dir
        self.fold = fold
        self.total_folds = total_folds
        self.writer = tf.summary.create_file_writer(self.log_dir)
        self.metrics_history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        with self.writer.as_default():
            for name, value in logs.items():
                tf.summary.scalar(f"{name}/fold_{self.fold}", value, step=epoch)
                
                # Store the metric value in our custom history
                if name not in self.metrics_history:
                    self.metrics_history[name] = []
                self.metrics_history[name].append(value)

                # If this is the last fold, calculate and log the average
                if self.fold == self.total_folds:
                    avg_value = np.mean(self.metrics_history[name])
                    tf.summary.scalar(f"{name}/average", avg_value, step=epoch)
                    
    def reset_states(self):
        self.metrics_history.clear()


def train(path_to_data, path_to_labels, window_size = 16, step_size =2/5, data_parts=4, epochs = 100, batch_size =10, early_stopping_patience = 25):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = int(step_size* window_size)
    print(step_sequence)

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)
    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    all_results = []
    
    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")

        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]

        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        # Create and compile model
        model = create_1dcnn_lstm_arch(input_shape=(train_d.shape[-2], train_d.shape[-1]))


        # Calculate the number of steps per epoch
        steps_per_epoch = len(train_d) // batch_size

        # Total decay steps (number of iterations over all epochs)
        total_decay_steps = steps_per_epoch * epochs
        # Cosine Decay Learning Rate Scheduler
        lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
            initial_learning_rate=0.0005,
            decay_steps=total_decay_steps,
            alpha=0.00001  # Minimum learning rate is 0
        )
        # Compile the model with the learning rate schedule
        optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
        model.compile(optimizer=optimizer, loss=correlation_coefficient_loss,
                      metrics=['mse', 'mae', correlation_coefficient_accuracy])

        # Callbacks
        tb_callback = CustomTensorBoardCallback(log_dir, fold + 1, data_parts)
        early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=early_stopping_patience, restore_best_weights = True)

        # Model checkpoint callback to save the best model
        best_model_path = os.path.join(run_dir, f'best_model_fold_{fold + 1}.keras')
        model_checkpoint = keras.callbacks.ModelCheckpoint(
            best_model_path,
            save_best_only=True,
            monitor='val_loss',
            mode='min',
        )

        # Train the model
        history = model.fit(train_d, train_lbs, batch_size=batch_size, epochs=epochs,
                            validation_data=(val_d, val_lbs), 
                            callbacks=[tb_callback, early_stopping, model_checkpoint])

        # Load the best model for evaluation
        model = keras.models.load_model(best_model_path, custom_objects={'correlation_coefficient_loss': concordance_correlation_coefficient_loss, 'correlation_coefficient_accuracy': correlation_coefficient_accuracy})

        # Evaluate model on validation data
        
        val_pred = model.predict(val_d, batch_size=batch_size)
        val_pred = val_pred.reshape(val_timesteps.shape)
        val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
        val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
        val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)

        # Evaluate model on test data
        test_pred = model.predict(test_d, batch_size=batch_size)
        test_pred = test_pred.reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)
        test_loss, test_mse, test_mae, test_acc = model.evaluate(test_d, test_lbs, batch_size=batch_size)

        # Store fold results
        fold_result = {
            'Fold': fold + 1,
            'val_flat_pearson_coefficient': val_prc_coef,
            'test_loss': test_loss,
            'test_mse': test_mse,
            'test_mae': test_mae,
            'test_pearson_coefficient': test_acc,
            'test_pearson_coefficient_flat': test_prc_coef
        }
        all_results.append(fold_result)

        fold_metrics.append(fold_result)

        # Save fold model
        #model.save(os.path.join(run_dir, f'model_fold_{fold + 1}.keras'))


    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
    
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    all_results.append(avg_metrics)

    # Create DataFrame and save to CSV
    results_df = pd.DataFrame(all_results)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)

    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    

    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")

if __name__ == "__main__":
    path = "/home/glenn/Downloads/"
    path  = "../DATA/"

    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=16,
        batch_size=64,
        epochs=150,
        data_parts=4
    )

2024-10-17 11:39:23.552578: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-17 11:39:23.695417: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-17 11:39:23.753425: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-17 11:39:23.771586: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-17 11:39:23.875788: I tensorflow/core/platform/cpu_feature_guar

6
Fold 1/4


/gpfs/home3/gdwildt/micromamba/envs/tens_thesis/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-10-17 11:39:35.659913: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38486 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:ca:00.0, compute capability: 8.0


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 256000, 64)     │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256000, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 25600, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 25600, 128)     │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 25600, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 6400, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 6400, 258)      │       198,402 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 6400, 258)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 1600, 258)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 1600, 258)      │       333,078 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1600, 258)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d               │ (None, 400, 258)       │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 400, 258)       │       533,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 400, 258)       │       533,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 400, 258)       │       533,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 400, 1)         │           259 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 400)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_12 (Conv1D)              │ (None, 256000, 64)     │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 256000, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_9 (MaxPooling1D)  │ (None, 25600, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_13 (Conv1D)              │ (None, 25600, 128)     │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 25600, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_10 (MaxPooling1D) │ (None, 6400, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_14 (Conv1D)              │ (None, 6400, 258)      │       198,402 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 6400, 258)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_11 (MaxPooling1D) │ (None, 1600, 258)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_15 (Conv1D)              │ (None, 1600, 258)      │       333,078 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 1600, 258)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d_3             │ (None, 400, 258)       │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 400, 258)       │       533,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 400, 258)       │       533,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 400, 258)       │       533,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 400, 1)         │           259 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 400)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,198,739 (8.39 MB)

 Trainable params: 2,198,739 (8.39 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/150
16/16 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - correlation_coefficient_accuracy: 0.1790 - loss: 0.8209 - mae: 0.3101 - mse: 0.1501 - val_correlation_coefficient_accuracy: 0.3791 - val_loss: 0.6159 - val_mae: 0.4410 - val_mse: 0.2585
Epoch 2/150
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - correlation_coefficient_accuracy: 0.3107 - loss: 0.6890 - mae: 0.4604 - mse: 0.2896 - val_correlation_coefficient_accuracy: 0.3789 - val_loss: 0.6163 - val_mae: 0.3085 - val_mse: 0.1419
Epoch 3/150
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - correlation_coefficient_accuracy: 0.3718 - loss: 0.6285 - mae: 0.3826 - mse: 0.2088 - val_correlation_coefficient_accuracy: 0.4254 - val_loss: 0.5709 - val_mae: 0.3985 - val_mse: 0.2104
Epoch 4/150
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - correlation_coefficient_accuracy: 0.4568 - loss: 0.5431 - mae: 0.3420 - mse: 0.1711 - val_correlation_coefficient_accuracy: 0.4653 - val_loss: 0.5308 - val_mae: 0.4683 - val_mse: 0.2640
Epoch 5/150
16/16 ━━━━━━━━━━━━━━━━━━━━ 

In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

2024-10-16 17:38:38.358120: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-16 17:38:38.465208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-16 17:38:38.513752: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-16 17:38:38.530372: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-16 17:38:38.610117: I tensorflow/core/platform/cpu_feature_guar

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
